In [14]:
%%capture
%pip install nltk transformers
%pip install sentencepiece

In [15]:
import nltk
from nltk.corpus import brown
from transformers import TFAlbertModel, AlbertTokenizer
import numpy as np
import csv
from nltk.tokenize import word_tokenize
import os
from tqdm import tqdm

import tensorflow as tf

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [17]:
nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\nawee\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nawee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
model_name = "albert-base-v2"
model = TFAlbertModel.from_pretrained(model_name)
tokenizer = AlbertTokenizer.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.dense.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [19]:
def embed_words(words):
    tokens = tokenizer.batch_encode_plus(words, padding='max_length', truncation=True, return_tensors='tf', max_length=512)
    outputs = model(tokens['input_ids'])
    embeddings = outputs.last_hidden_state
    averaged_embeddings = tf.reduce_mean(embeddings, axis=1)
    return averaged_embeddings

In [20]:
dataset_path = r"Coursework Brief/Reuters-21578/Reuters-21578/reuters/reuters/reuters/training"
dataset_dirs = os.listdir(dataset_path)

In [21]:
data = []
for i in tqdm(dataset_dirs):
    with open(f"{dataset_path}/{i}", 'r') as f:
        content = f.read()
        data.append(content)

  0%|          | 0/7769 [00:00<?, ?it/s]

100%|██████████| 7769/7769 [00:01<00:00, 5106.56it/s]


In [22]:
words = []
for text in data:
  temp_words = word_tokenize(text)
  words.append(temp_words)

In [23]:
from itertools import chain

words_flattened = list(chain(*words))


In [24]:
len(words_flattened)

1135633

In [25]:
words_flattened = [str(element) for element in words_flattened]

In [26]:
embeddings = embed_words(words_flattened)

ResourceExhaustedError: Exception encountered when calling layer 'embeddings' (type TFAlbertEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[1135633,512,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ResourceGather] name: 

Call arguments received by layer 'embeddings' (type TFAlbertEmbeddings):
  • input_ids=tf.Tensor(shape=(1135633, 512), dtype=int32)
  • position_ids=None
  • token_type_ids=tf.Tensor(shape=(1135633, 512), dtype=int32)
  • inputs_embeds=None
  • past_key_values_length=0
  • training=False

In [ ]:
# Save embeddings and metadata in TSV format
tsv_file = 'word_embeddings.tsv'
metadata_file = 'metadata.tsv'

# Save embeddings
np.savetxt(tsv_file, embeddings, delimiter='\t')

# Save metadata
with open(metadata_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerows([[word] for word in words_flattened])

In [ ]:
len(embeddings)

5000